# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [8]:
# import libraries
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [10]:
# load data from database
engine = create_engine('sqlite:///Clean_dataset.db')
df = pd.read_sql_table('Data_Table', con=engine)
X = df['message']
Y = df.drop(['message', 'genre', 'id', 'original'], axis = 1)


### 2. Write a tokenization function to process your text data

In [11]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    # Return the list of tokens
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

# Create the pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Train the pipeline
pipeline.fit(X_train, y_train)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [14]:
pipeline.get_params()


{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f2d1d663598>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [15]:
y_pred = pipeline.predict(X_test)
accuracy = (y_pred == y_test).mean()
print("Accuracy:", accuracy)
total_acc=(y_pred == y_test).mean().mean()
total_acc

Accuracy: related                   0.785660
request                   0.876049
offer                     0.995042
aid_related               0.741228
medical_help              0.922387
medical_products          0.950992
search_and_rescue         0.977498
security                  0.982647
military                  0.970824
child_alone               1.000000
water                     0.947941
food                      0.929825
shelter                   0.920290
clothing                  0.986842
money                     0.981693
missing_people            0.988558
refugees                  0.967963
death                     0.958619
other_aid                 0.868421
infrastructure_related    0.937643
transport                 0.955950
buildings                 0.950801
electricity               0.977498
tools                     0.993898
hospitals                 0.991228
shops                     0.995805
aid_centers               0.987223
other_infrastructure      0.957094
weather_re

0.94271230612763801

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
from sklearn.metrics import classification_report

# Assuming you have trained the pipeline and have predictions stored in 'predictions'
predictions = pipeline.predict(X_test)

# Iterate through each output category
for i, column in enumerate(Y.columns):
    print("Category:", column)
    print(classification_report(y_test.iloc[:, i], predictions[:, i]))
    print("--------------------------------------")




Category: related
             precision    recall  f1-score   support

          0       0.61      0.37      0.46      1266
          1       0.82      0.93      0.87      3938
          2       0.75      0.15      0.25        40

avg / total       0.77      0.79      0.76      5244

--------------------------------------
Category: request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      4349
          1       0.82      0.35      0.49       895

avg / total       0.87      0.88      0.85      5244

--------------------------------------
Category: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244

--------------------------------------
Category: aid_related
             precision    recall  f1-score   support

          0       0.73      0.89      0.80      3113
    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
from sklearn.model_selection import GridSearchCV

# Define the parameters grid
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'clf__estimator__n_estimators': [10, 50]  # Update parameter name to 'clf__estimator__n_estimators'
}

# Create the GridSearchCV object
cv = GridSearchCV(pipeline, param_grid=parameters)

# Fit the GridSearchCV object to the training data
cv.fit(X_train, y_train)

# Print the best parameters
print(cv.best_params_)







{'clf__estimator__n_estimators': 50, 'vect__ngram_range': (1, 2)}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
new_predictions = pipeline.predict(X_test)

# Iterate through each output category
for i, column in enumerate(Y.columns):
    print("Category:", column)
    print(classification_report(y_test.iloc[:, i], new_predictions[:, i]))
    print("--------------------------------------")




Category: related
             precision    recall  f1-score   support

          0       0.61      0.37      0.46      1266
          1       0.82      0.93      0.87      3938
          2       0.75      0.15      0.25        40

avg / total       0.77      0.79      0.76      5244

--------------------------------------
Category: request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      4349
          1       0.82      0.35      0.49       895

avg / total       0.87      0.88      0.85      5244

--------------------------------------
Category: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244

--------------------------------------
Category: aid_related
             precision    recall  f1-score   support

          0       0.73      0.89      0.80      3113
    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [20]:
import pickle

# Save the model to a file
filename = 'model.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

print("Model saved as", filename)

Model saved as model.pkl


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.